## Data processing

In [2]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o data/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  2238k      0 --:--:-- --:--:-- --:--:-- 2245k


In [3]:
with open("data/input.txt") as f:
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("howdy there."))
print(decode(encode("howdy there.")))

[46, 53, 61, 42, 63, 1, 58, 46, 43, 56, 43, 8]
howdy there.


In [6]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:20])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56])


In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

`block_size + 1` is due to fact that there will be `block_size` examples within this block. In the case above, the examples are:

- `18 -> 47`
- `18, 47 -> 56`
- ...

where `->` denotes that the lhs is used to "predict" the rhs, so above example contains 8 examples, as specified by `block_size`.

In [9]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"context: {context} and target: {target}")

context: tensor([18]) and target: 47
context: tensor([18, 47]) and target: 56
context: tensor([18, 47, 56]) and target: 57
context: tensor([18, 47, 56, 57]) and target: 58
context: tensor([18, 47, 56, 57, 58]) and target: 1
context: tensor([18, 47, 56, 57, 58,  1]) and target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]) and target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target: 58


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is [24] the target: 43
When input is [24, 43] the target: 58
When input is [24, 43, 58] the target: 5
When input is [24, 43, 58, 5] the target: 57
When input is [24, 43, 58, 5, 57] the target: 1
When input is [24, 43, 58, 5, 57, 1] the target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
When input is [44] the target: 53
When input is [44, 53] the target: 56
When input is [44, 53, 56] the target: 1
When input is [44, 53, 56, 1] the target: 58
When input is [44, 53, 56, 1, 58] the target: 46
When input is [44, 53, 56,

## Transformer model

### Bigram model

In [28]:
import torch
from torch import nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embeding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # only focusing on last step/element
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            # (B, 1) for each batch have 1 prediction
            idx_next = torch.multinomial(probs, num_samples=1)
            # (B, T + 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [30]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [39]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch("train")
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.400357961654663


In [41]:
print(decode(m.generate(idx=idx, max_new_tokens=300)[0].tolist()))


TENTUSBRirce htate hy adfouliepastrou hour tonghou
S:
Fiovattilis usere g thartet thalire d rth?
Ay! s,
wodollltalrr,
ORE:
Nonge yif Sleng nofuco mare: theve, hoo eg sth nd spe; acourinuof in onceshachr sesmf r, ithern IINGe thes; ovin, macon oto st
OForitourthe He, ispth athilire arethofand bor que
